## Imports

In [85]:
import torch
from torch import nn

## Model Class

In [86]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE,self).__init__()
        
        
        self.common_fc = nn.Sequential(
            nn.Linear(28*28, out_features=196), nn.Tanh(),
            nn.Linear(196, out_features=48), nn.Tanh(),
        )
        
        self.mean_fc = nn.Sequential(
            nn.Linear(48, out_features=16), nn.Tanh(),
            nn.Linear(16, out_features=2), nn.Tanh()
        )
        
        self.log_var_fc = nn.Sequential(
            nn.Linear(48, out_features=16), nn.Tanh(),
            nn.Linear(16, out_features=2), nn.Tanh()
        )
        
        self.decoder_fcs = nn.Sequential(
            nn.Linear(2, out_features=16), nn.Tanh(), 
            nn.Linear(16, out_features=48), nn.Tanh(),
            nn.Linear(48, out_features=196), nn.Tanh(),
            nn.Linear(196, out_features=28*28), nn.Tanh(),
        )
        

    def encode(self,x):
        # B,C,H,W
        out = self.common_fc(torch.flatten(x, start_dim=-1))
        mean = self.mean_fc(out)
        log_var = self.log_var_fc(out)
        return mean, log_var


    def sample(self, mean, log_var):
        std = torch.exp(0.5*torch.flatten(log_var, start_dim=-1))
        z = torch.randn_like(torch.flatten(std, start_dim=-1))
        return z * std + mean
    
    
    def decode(self, z):
        flat_z = torch.flatten(z, start_dim=-1)
        out = self.decoder_fcs(flat_z)
        out = torch.reshape(out, [1, 28*28])
        return out
        
    
    def forward(self, batch_x):
        #B,C,H,W
        outputs = []
        for sample in batch_x:
            #Encoder
            mean, log_var = self.encode(torch.flatten(sample))
            #Sampling
            z = self.sample(mean,log_var)
            #Decoder
            outputs.append(self.decode(z))

        out = torch.stack(outputs, dim=1)
        return mean, log_var, out


## Training Parameters

In [87]:
# Device init
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


LEARN_RATE = 1e-3

## Data Creation

In [88]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.v2 as transforms

def create_data():
    # ---INITIALIZE DATASET ---
    #Convert pilimage dataset to a standart numpy dataset
    dataset = MNIST(
        root='./data',
        download=True,  # Add this to download the dataset if needed
        transform= transforms.ToTensor()
    )

    TRIM_LEN = int(58_000)  # 60,000 - 58,000 = 2,000 SAMPLES
    TRAIN_PORTION = 0.9 # 90% training 10% everything else
    TRAIN_LEN = int((len(dataset) - TRIM_LEN) * TRAIN_PORTION)
    
    # ---SPLIT DATASET---
    train_ds, test_ds, _ = random_split(
        dataset,  # Split the dataset, not the dataloader!
        [TRAIN_LEN, len(dataset) - TRIM_LEN - TRAIN_LEN, TRIM_LEN]
    )
    #print(f"train length: {len(train_ds)} test_length: {len(test_ds)}")
    
    # ---CREATE DATALOADERS from the split datasets---
    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)
    
    return train_loader, test_loader

## Training

In [ ]:
def train_vae(model, train_loader):
    # Method 1: One-liner
    single_batch = next(iter(train_loader))[0].to(device)


    # ---------Feed Forward---------

    # Extract just the generated images for now
    _,_,img_gen_batch = model.forward(single_batch)


    #---------Back Prop---------

    # Accumulate model loss
    for ds_sample, img_gen in zip(single_batch, img_gen_batch[0]):
        flat_sample = torch.flatten(ds_sample)
        torch.kl_div(img_gen, flat_sample)


In [90]:
# Initialize the model
my_vae = VAE().to(device)
optimizer = torch.optim.Adam(params=my_vae.parameters(),lr = LEARN_RATE)


train_loader,test_loader = create_data()
train_vae(model=my_vae, train_loader=train_loader)

single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
single_batch's shape:  torch.Size([784])
